### PyTorch实现线性回归

1.生成训练数据

In [10]:
import torch

# 确保CUDA可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 生成数据
inputs = torch.rand(100, 3) # 随机生成shape为(100,3)的tensor，里边每个元素的值都是0-1之间
weights = torch.tensor([[1.1], [2.2], [3.3]]) #预设的权重
bias = torch.tensor(4.4) #预设的bias
targets = inputs @ weights + bias + 0.1*torch.randn(100, 1) #增加一些误差，模拟真实情况

上边的代码首先随机生成100条数据，每条数据都有3个feature。具体值是随机产生的0-1之间的值。接着通过我们人为设定的weight和bias，对随机生成的inputs经过线性变化，再加上一些小的随机误差。生成对应的targets作为label值。 我们希望线性回归模型，通过inputs和targets进行训练。最终能调整训练的权重和偏置参数，达到和我们人为设置的weights和bias接近的值

2.初始化线性回归的参数

In [11]:
# 初始化参数时直接放在CUDA上，并启用梯度追踪
w = torch.rand((3, 1), requires_grad=True, device=device)
b = torch.rand((1,), requires_grad=True, device=device)

3.进行训练

In [12]:
# 将数据移至相同设备
inputs = inputs.to(device)
targets = targets.to(device)

#设置超参数
epoch = 10000
lr = 0.003

for i in range(epoch):
    outputs = inputs @ w + b
    loss = torch.mean(torch.square(outputs - targets))
    print("loss:", loss.item())

    loss.backward()

    with torch.no_grad(): #下边的计算不需要跟踪梯度
        w -= lr * w.grad
        b -= lr * b.grad

    # 清零梯度
    w.grad.zero_()
    b.grad.zero_()

print("训练后的权重 w:", w)
print("训练后的偏置 b:", b)

loss: 59.025115966796875
loss: 57.72413635253906
loss: 56.451908111572266
loss: 55.207786560058594
loss: 53.99116134643555
loss: 52.80142593383789
loss: 51.63796615600586
loss: 50.50023651123047
loss: 49.38763427734375
loss: 48.29962921142578
loss: 47.23565673828125
loss: 46.19519805908203
loss: 45.177734375
loss: 44.18274688720703
loss: 43.20975112915039
loss: 42.25825500488281
loss: 41.327781677246094
loss: 40.41787338256836
loss: 39.52806854248047
loss: 38.65792465209961
loss: 37.807010650634766
loss: 36.97489929199219
loss: 36.16117477416992
loss: 35.36543273925781
loss: 34.58727264404297
loss: 33.8263053894043
loss: 33.0821533203125
loss: 32.35444641113281
loss: 31.6428165435791
loss: 30.94691276550293
loss: 30.266386032104492
loss: 29.600893020629883
loss: 28.95010757446289
loss: 28.31369972229004
loss: 27.691356658935547
loss: 27.082759857177734
loss: 26.48761749267578
loss: 25.90561866760254
loss: 25.336483001708984
loss: 24.77992057800293
loss: 24.235658645629883
loss: 23.7034

上边的代码，首先必须将inputs和targets也移动到和参数同样的设备上，不同设备上的tensor是无法进行计算的。 然后我们设置超参数迭代次数epoch，和学习率lr。 然后进入训练循环，我们只需要定义前向传播过程就可以，最后调用loss的backward方法。PyTorch会根据自动生成的计算图进行反向传播和梯度计算。后边我们就可以用w.grad来获取loss对w的偏导数了。 然后对参数进行梯度更新，需要注意这个更新计算不记录梯度，所以放在torch.no_grad()的上下文里。 还需要注意的是，需要对更新后的参数的梯度进行清零，不然梯度值会一直累积